## Summary
     In this notebook, data preparation steps are cover. 
        - From descriptive stats, it was found that are negative values - Excluded values which are negative 
        - Nulls in certain columns  - drop these records as number was less
        - Columns need proper formatting - will be easier to manage dot operations in pandas
        - Save the file as csv for further analysis

### Step 1. Let us look on the datasets for validating the quality of data and understanding it

In [14]:
## load the libraries for computation
import pandas as pd
import numpy as np

# Load the libraries for plotting
import seaborn as sns 
import matplotlib.pyplot as plt

In [15]:
# load the data 
df =pd.read_excel("Imaginary Auto Company.xlsx")
del df['Unnamed: 0'] # delete the noise
print("This is the shape of the data", df.shape)
df.head(1)  # look on one record of dataset

This is the shape of the data (80493, 28)


,booking to delivery date,unit price($000),type of sale,date of sale,mode of transport,no of units,cust_group_name,cust_segment_name,Customer Id,delivery_month_from,...,plant_country_name,most expensive part_l1,most expensive part_l2,shipto_city,shipto_nr,soldto_city,soldto_nr,car_nr,shipto_country,soldto_country
0,long,1662.54,retail,2159-09-26,Truck,2640.0,aa,Prime,953.0-952.0,2160-04-01,...,Philippines,bonnet,bonner,Algeria,953,Algeria,952,Pag99992258,Algeria,Algeria


In [16]:
## we need to rename the colums are they are not in proper format
## we can use dictiotary to map the names with selected columns
df.rename(columns={'booking to delivery date': 'booking_to_delivery_date',
                   'unit price($000)': 'unit_price_inK',
                  'type of sale': 'type_of_sale',
                   'date of sale': 'date_of_sale',
                  'mode of transport': 'mode_of_transport',
                  'no of units': 'no_of_units',
                   'Customer Id': 'Customer_Id', 
                  'most expensive part_l1': 'most_expensive_part_l1',
                  'most expensive part_l2' : "most_expensive_part_l2"}, inplace=True)

In [17]:
df.describe().T  # let us looks on the stats properties and transpose the matrix
##  There are negative values in unit_price_inK and no_of_units, 

,count,mean,std,min,25%,50%,75%,max
unit_price_inK,80493.0,2600.815817,11149.798374,-31282.02,634.1775,1348.6,2447.06,542006.08
no_of_units,80493.0,2257.537866,5868.182337,-646.80,287.1880,660.0,1999.80,184800.00
plant_nr,80493.0,1028.360926,29.225280,1001.00,1009.0000,1009.0,1054.00,1085.00
shipto_nr,80493.0,3612.257836,1056.959786,952.00,4132.0000,4139.0,4189.00,5361.00
soldto_nr,80493.0,3614.924540,1071.037352,952.00,4132.0000,4140.0,4206.00,5363.00


#### Negative Values  Treatment
    We are excluding the noise from the data that is introduced by negative values in the unit price and no, of units columns

In [18]:
## remove the data which has issues 
df1 = df[(df['unit_price_inK'] > 0)  & (df['no_of_units'] > 0)] 

# as confirmed with team, it is safe to remove as they are just noise in the data
## we have negavtive values in the sales price. 
## we need to drop these 
df1.describe().T ## validate if negative values are removed

,count,mean,std,min,25%,50%,75%,max
unit_price_inK,69651.0,2571.733560,10544.106272,0.220,663.789445,1393.040,2497.0,542006.08
no_of_units,69651.0,2479.914791,5932.205166,0.022,458.326000,916.674,2200.0,184800.00
plant_nr,69651.0,1028.022641,28.832463,1001.000,1009.000000,1009.000,1045.0,1085.00
shipto_nr,69651.0,3632.723234,1037.480174,952.000,4132.000000,4140.000,4190.0,5361.00
soldto_nr,69651.0,3631.151398,1051.798947,952.000,4132.000000,4140.000,4203.0,5363.00


### Treating missing data
    We have two options
        a. Exclude the datasets which has missing values
        b. Train and build seperate model for missing featureset data

As of now, I am using option a. and will drop them from the dataset as of now. 

NOTE - We can find a better appoach to fill Nulls with the help of SMEs.

In [19]:
## let us look on the columns which are having missing values 
cols_with_null = df.columns[df.isnull().any()] # we have list of the colums which are having null 

# create a dataframe of these columns for better understanding and take a decision what to do with Null values
df_cols_null_value = df[cols_with_null]
print(df_cols_null_value.info())
df_cols_null_value.head(2)
## Decision: missing values are not very critical,we can drop them from the dataframe itself

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80493 entries, 0 to 80492
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   cust_segment_name       80473 non-null  object
 1   plant_city              80447 non-null  object
 2   plant_country_name      80447 non-null  object
 3   most_expensive_part_l1  80473 non-null  object
 4   most_expensive_part_l2  80451 non-null  object
 5   shipto_city             80331 non-null  object
 6   soldto_city             80466 non-null  object
dtypes: object(7)
memory usage: 4.3+ MB
None


,cust_segment_name,plant_city,plant_country_name,most_expensive_part_l1,most_expensive_part_l2,shipto_city,soldto_city
0,Prime,Quezon City,Philippines,bonnet,bonner,Algeria,Algeria
1,Prime,Quezon City,Philippines,bonnet,bonner,Algeria,Algeria


In [20]:
df1.dropna(axis=0, inplace=True) # drop the nulls 

df1.isna().sum() # missing values are treated 

<ipython-input-20-eb8eff78f944>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(axis=0, inplace=True) # drop the nulls


booking_to_delivery_date    0
unit_price_inK              0
type_of_sale                0
date_of_sale                0
mode_of_transport           0
no_of_units                 0
cust_group_name             0
cust_segment_name           0
Customer_Id                 0
delivery_month_from         0
delivery_month_to           0
type                        0
item_class_l1               0
item_class_l2               0
item_class_l3               0
colour                      0
plant_nr                    0
plant_city                  0
plant_country_name          0
most_expensive_part_l1      0
most_expensive_part_l2      0
shipto_city                 0
shipto_nr                   0
soldto_city                 0
soldto_nr                   0
car_nr                      0
shipto_country              0
soldto_country              0
dtype: int64

In [21]:
## are there duplicates in the dataset?
## are there duplicate values 
print(" Shape of dataframe original", df1.shape)

# drop duplicates 
df1 = df1.drop_duplicates()

## are there duplicate values 
print(" Shape of dataframe after duplicate drop", df1.shape)

 Shape of dataframe original (69402, 28)
 Shape of dataframe after duplicate drop (46125, 28)


In [22]:
## Save the dataset for further analysis

In [23]:
df1.to_csv("step1_prepared_data.csv")

In [24]:
df1.describe()

,unit_price_inK,no_of_units,plant_nr,shipto_nr,soldto_nr
count,46125.000000,46125.000000,46125.000000,46125.000000,46125.000000
mean,2643.188284,2572.755295,1026.814504,3599.396054,3592.635447
std,10838.148132,6443.660392,28.849672,1059.480663,1080.655713
min,0.220000,0.022000,1001.000000,952.000000,952.000000
25%,702.973590,440.000000,1009.000000,2639.000000,2639.000000
50%,1421.420000,878.680000,1009.000000,4140.000000,4141.000000
75%,2559.520656,2200.000000,1045.000000,4192.000000,4210.000000
max,542006.080000,184800.000000,1085.000000,5361.000000,5363.000000
